In [1]:

import os
import pandas as pd
import numpy as np
import math

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import gc




#### Variables Globales 

In [2]:
#### Variables Globales 


# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg

temps_serie = 5  # en min
temps_traitement = 10 # en min
temps_serie_courant = temps_serie
temps_gs = 4 # en min


Mgmax, Mgmin, Mglimite = 0.045, 0.035, 0.03
AjoutPoche, DemandePoche = False, False

# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg


#### Fonction Globales 

In [3]:
# Pour calculer la longueur du fil fourré 
def calcul_quantite_mg(P,S,t,e,T,R,Mg,K) :
    """
    Calcule la Quantité d'alliage au magnésium (en Kg) à introduire dans la fonte pour obtenir du graphite spherodial.
    Args:
    P: Poids de fonte à traiter en Kg.
    S: Taux de souffre de la fonte de base en %.
    t: Temps de séjour en minutes prévu pour la fonte après traitement.
    T: Température (degrés Celsius) de la fonte au moment du traitement, mesurée au couple.
    R: Rendement en magnésium de l'opération en %.
    Mg: Taux en magnésium dans l'alliage en %.
    K: Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal en %.

    Returns:
    Q: Quantité d'alliage au magnésium à utiliser en Kg.
    """
    Q = P * (0.76 * (S - 0.01) + K + t * e) * (T / 1450) ** 2 / (R * Mg / 100)

    return Q

def calcul_longueur_fil( PPT, S, TPT,
    R, masse_fil, masse_mg_fil, Mgmax, eP, 
    PFClimite, Mglimite ):

    # % de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    K = ( Mgmax*(PFClimite + PPT) - Mglimite*PFClimite )/ PPT

    # Masse de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    Mgfil = masse_mg_fil/masse_fil *100
    Q = calcul_quantite_mg(PPT,S,temps_gs,eP,TPT,R,Mgfil,K) # en Kg


    # Longueur de fil pour avoir la masse de Mg manquante
    L = Q / (masse_fil* 1e-3)   # en m

    return K, L




# Pour le passage d'un etat à l'autre (consom, Serie, Panne, Fin)
def PendantChangement_serie ( Mg, PFC) :
    # Variables dans ce programme
    global temps_serie, temps_serie_courant

    # pendant Temps_Serie :
    Mg -= eC # perte de Mg par minute ou par seconde
    temps_serie_courant -= 1 # pas de temps en minute
    
    etat_suivant = "etat_Serie"
    if temps_serie_courant == 0:
        temps_serie_courant = temps_serie  # Réinitialiser le temps de série
        
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        etat_suivant = "etat_Consom"
    
    return Mg, PFC, etat_suivant

def PendantConsommation ( Mg, PFC):
    
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire


    # On produit les moules !! 
    masse_grappe_i = liste_masses_grappes_moules[0]
    cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
    quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
    
    # La cadence de consommation en kg/min du i-ème modèle
    cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
    

    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC -= cadence_fonte_i
    Mg -= eC


    # On reste dans l'état consom si il reste des moulles à faire
    etat_suivant= "etat_Consom"

    # Si on a fini de tout produire alors on stoppe la procédure 
    # On passe à l'état Fin
    if not liste_masses_grappes_moules:
        etat_suivant = "etat_Fin"
    
    return Mg, PFC, etat_suivant

def PendantPanne ( Mg, PFC) :
    # Variables dans ce programme
    global Panne

    # pendant Temps_Panne non définis :
    Mg -= eC # perte de Mg par minute ou par seconde


    etat_suivant = "etat_Panne"
    if not Panne :
        etat_suivant = "etat_Consom"
    
    return Mg, PFC, etat_suivant



def gerer_etat(Mg, PFC, etat_courant):
    """
    Gère les transitions entre différents états et effectue les opérations associées.

    Parameters:
        etat_courant (str): L'état actuel du système ("etat_Consom", "etat_Serie", "etat_Fin").
        Mg (float): La quantité de matériau Mg.
        PFC (float): La quantité de PFC disponible.

    Returns:
        Tuple: Retourne les variables mises à jour pour Mg, PFC, etat_suivant.
    """

    # Constantes dans ce programme
    global running


    # Gérer l'état de consommation
    if etat_courant == "etat_Consom":
        Mg, PFC, etat_suivant = PendantConsommation ( Mg, PFC)

    # Gérer l'état de série
    elif etat_courant == "etat_Serie":
        Mg, PFC, etat_suivant = PendantChangement_serie ( Mg, PFC)

    # Gérer l'état de panne
    elif etat_courant == "etat_Panne":
        Mg, PFC, etat_suivant = PendantPanne ( Mg, PFC)

    # Gérer l'état de fin
    elif etat_courant == "etat_Fin":
        running = False  # Fin du processus
        etat_suivant = "etat_Fin"
    return Mg, PFC, etat_suivant


def gerer_poche(Mg, PFC):
    """
    Gère l'ajout de la poche. Si Poche est True, met à jour Mg et PFC.
    
    Args:
    Mg (float): La quantité actuelle de Mg.
    PFC (float): La quantité actuelle de Fonte.

    
    Returns:
    Mg (float): La nouvelle quantité de Mg.
    PFC (float): La nouvelle quantité de fonte.
    """

    # Constantes dans ce programme
    global Mgmax, poche_i
    # Variables dans ce programme
    global AjoutPoche

    if AjoutPoche:
        Mg = Mgmax
        PFC += poche_i
        AjoutPoche = False
    return Mg, PFC




In [4]:
# Pour calculer le temps du prochain Traitement GS
def Calcule_Temps_prochain_traitement(delai_avt_traitement,heure_lancement):
    # Trouver les positions des unités de temps dans la chaîne
    pos_hr = heure_lancement.find('hr')
    pos_min = heure_lancement.find('min')
    pos_sec = heure_lancement.find('s')

    # Extraire les valeurs numériques
    now_hours = int(heure_lancement[:pos_hr].strip()) 
    now_minutes = int(heure_lancement[pos_hr+3:pos_min].strip()) 
    now_seconds = int(heure_lancement[pos_min+4:pos_sec].strip()) 


    if delai_avt_traitement < 0 :
        print("Lancer le Traitement GS !")

    # Calculer le temps total en secondes
    total_seconds = now_hours * 3600 + now_minutes * 60 + now_seconds + delai_avt_traitement*60

    # Calculer les nouvelles heures, minutes et secondes
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)

    heure_prochain_lancement = f"{hours} hr {minutes} min {seconds} s"
    return heure_prochain_lancement    


# Calcule du temps limite avant lancement prochain traitement
def calculer_temps_epuis_PFC(PFC):
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, temps_serie
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire, temps_serie

    # Calcul de la fonte four consommable
    PFCconsommable = PFC - PFCmin

    # Initialisation du temps total
    temps_total = 0

    # Boucle sur chaque modèle de moules
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min
        
        # Fonte nécessaire pour produire tous les moules de ce modèle
        fonte_necessaire_i = quantite_moules_a_produire_i * masse_grappe_i

        if PFCconsommable >= fonte_necessaire_i:
            # Si la fonte est suffisante pour produire tous les moules
            temps_pour_produire_i = fonte_necessaire_i / cadence_fonte_i
            temps_total += temps_pour_produire_i
            PFCconsommable -= fonte_necessaire_i

            # Ajouter le temps de la série s'il y a d'autres modèles à produire
            if i < len(liste_masses_grappes_moules) - 1:
                temps_total += temps_serie
        else:
            # Si la fonte n'est pas suffisante, calcul du temps possible avec la fonte restante
            temps_possible = PFCconsommable / cadence_fonte_i
            temps_total += temps_possible
            break  # On arrête la boucle, car il n'y a plus de fonte

    return temps_total

def calcul_temps_limite(PFC, Mg):
    
    global Mgmin, eC, temps_traitement

    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre la fonte minimal (en Kg) dans le four de coulée
    temps_epuis_PFC = calculer_temps_epuis_PFC(PFC)
    delai_avt_traitement_fonte_four = temps_epuis_PFC - temps_traitement


    
    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre le pourcentage minimal (%) dans le four de coulée
    Mgconsommable = Mg - Mgmin
    temps_epuis_Mg = Mgconsommable/eC
    delai_avt_traitement_mg_four = temps_epuis_Mg - temps_traitement


    # Temps en minute avant de lancer le traitement (Du four de fusion au four de coulée)
    delai_avt_traitement = min(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)
    tempslimite = temps_traitement + delai_avt_traitement
    # mise a jour de la masse mg avant ajout poche après consomation de mg
    Mgconsommer = tempslimite*eC
    Mgconsommer = (temps_traitement + delai_avt_traitement)*eC
    Mglimite = Mg - Mgconsommer



    # Mise à jour de la masse de fonte avant ajout dans le four après consommation pendant
    # temps_traitement + delai_avt_traitement
    PFCconsommer = 0
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min

        # Vérification du temps restant par rapport au temps nécessaire pour produire tous les moules de ce modèle
        temps_pour_produire_i = quantite_moules_a_produire_i / (cadence_fonte_i / masse_grappe_i)
        
        if tempslimite > temps_pour_produire_i:
            # Si le délai est supérieur au temps nécessaire, consommer toute la fonte pour ce modèle
            PFCconsommer += cadence_fonte_i * temps_pour_produire_i
            tempslimite -= temps_pour_produire_i

            if i < len(liste_masses_grappes_moules) - 1:
                tempslimite -= temps_serie
        else:
            # Sinon, consommer la fonte en fonction du temps restant
            PFCconsommer += cadence_fonte_i * tempslimite
            break

    PFClimite =  PFC - PFCconsommer


    return delai_avt_traitement, Mglimite, PFClimite




# def gerer_Prevention(Mg, PFC, etat_courant):

#     delai_avt_traitement, Mglimite, PFClimite = calcul_temps_limite(PFC, Mg)
#     if delai_avt_traitement == 0 :

#     return Mg, PFC, etat_suivant



In [5]:
import plotly.graph_objs as go

def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig1 = go.FigureWidget()
    scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")

    fig2 = go.FigureWidget()
    scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
    fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

    return fig1, scatter1, fig2, scatter2

fig1, scatter1, fig2, scatter2 = create_figures()

#### Fonctions interactives 

##### Sans bonnes couleurs 

In [6]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import time


def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC

    global Poche, poche_i

    running = False
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    Mgdata = []
    t = initial_t
    etat_courant = "etat_Consom"  
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC

def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig1 = go.FigureWidget()
    scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")

    fig2 = go.FigureWidget()
    scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
    fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

    return fig1, scatter1, fig2, scatter2

def update_data():
    """Fonction de mise à jour des données."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC
    global Poche, poche_i, Mgmax
    
    while running:
        # Mettre à jour les données de temps
        timedata.append(t)
        
        # Appeler la fonction pour gérer l'état
        Mg, PFC, etat_suivant = gerer_etat(Mg, PFC, etat_courant)

        # Si l'on ajoute de la poche
        Mg, PFC = gerer_poche(Mg, PFC)

        
        # Mettre à jour les données pour les graphiques
        Mgdata.append(Mg)
        PFCdata.append(PFC)

        with fig1.batch_update():
            scatter1.update(x=timedata, y=Mgdata)
        
        with fig2.batch_update():
            scatter2.update(x=timedata, y=PFCdata)
            
        t += time_step
        etat_courant = etat_suivant
        
        # Pause pour simuler le temps entre les mises à jour
        time.sleep(time_step)

def start_plot(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running
    running = True

    global etat_courant, Panne
    Panne = False
    etat_courant = "etat_Consom"


    # # Démarrer la mise à jour des données dans un nouveau thread ou processus
    import threading
    thread = threading.Thread(target=update_data)
    thread.start()


def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_courant, Mg, PFC
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_courant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    with fig1.batch_update():
        scatter1.update(x=timedata, y=Mgdata)
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)

def serie_plot(button):
    """Fonction pour passer à l'état de série."""
    global etat_courant
    etat_courant = "etat_Serie"


def panne_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Panne
    Panne = True
    etat_courant = "etat_Panne"

def poche_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Poche
    Poche = True
    etat_courant = "etat_Consom"


def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False


def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start")
    reset_button = widgets.Button(description="Reset")
    stop_button = widgets.Button(description="Stop")
    poche_button = widgets.Button(description="Poche")
    serie_button = widgets.Button(description="Série")
    panne_button = widgets.Button(description="Panne")
    poche_button = widgets.Button(description="Poche")



    start_button.on_click(start_plot)
    reset_button.on_click(reset_plot)
    stop_button.on_click(stop_plot)
    poche_button.on_click(poche_plot)
    serie_button.on_click(serie_plot)
    panne_button.on_click(panne_plot)


    display(widgets.HBox([start_button, reset_button, stop_button, poche_button, serie_button, panne_button]), fig1, fig2)

def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig1, scatter1, fig2, scatter2
    fig1, scatter1, fig2, scatter2 = create_figures()
    create_buttons()

# Appel de la fonction principale pour démarrer l'application
main()

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '5311d429-d038-4e70-a184-ed76a8d94914'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': 'ef9300bc-9d29-4907-a514-28550e96d437'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})

##### Avec bonnes couleurs 

In [7]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import time
from threading import Timer

segments = []


message_output = widgets.Output()

def initialize_message():
    """Initialise le widget Output avec un fond blanc et pas de message."""
    with message_output:
        message_output.clear_output()
        display(widgets.HTML(
            "<div style='background-color:white; color:black; font-size:20px; padding:10px; text-align:center;'>"
            "Aucun message</div>"
        ))

def update_message(message):
    """Met à jour le message affiché avec un style personnalisé."""
    with message_output:
        message_output.clear_output()  # Efface les anciens messages
        display(widgets.HTML(
            f"<div style='background-color:yellow; color:red; font-size:24px; font-weight:bold; "
            f"border:2px solid black; padding:10px; text-align:center;'>"
            f"{message}</div>"
        ))

def Demandepoche(PFC, Mg) :
    global DemandePoche, AjoutPoche

    if DemandePoche :
        delai_avt_traitement, Mglimite, PFClimite = calcul_temps_limite(PFC, Mg)
        message = f"Lancer la poche dans {delai_avt_traitement} minutes"
        update_message(message)
        if AjoutPoche :
            DemandePoche = False
    return


def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC

    running = False
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    Mgdata = []
    t = initial_t
    etat_courant = "etat_Consom"
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC

def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig1 = go.FigureWidget()

    # Un seul segment pour Mg, avec la couleur dynamique
    scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    
    fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")
    
    fig2 = go.FigureWidget()
    scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
    fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

    return fig1, scatter1, fig2, scatter2

import asyncio


# async def update_data():
def update_data():
    """Fonction de mise à jour des données."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC
    global Mgmax, Mgmin, Mglimite
    global segments

    while running:

        if t == 0 :
            timedata.append(0)
            Mgdata.append(0.045)
        else :
            Demandepoche(PFC, Mg)
            Mg, PFC, etat_suivant = gerer_etat(Mg, PFC, etat_courant)
            Mg, PFC = gerer_poche(Mg, PFC)
            timedata.append(t)
            etat_courant = etat_suivant
            Mgdata.append(Mg)
            PFCdata.append(PFC)

            with fig2.batch_update():
                scatter2.update(x=timedata, y=PFCdata)

            x0, x1 = timedata[-2], timedata[-1]
            y0, y1 = Mgdata[-2], Mgdata[-1]

            if y0 > Mgmin and y1 > Mgmin:
                color = 'green'
            elif y0 < Mglimite and y1 < Mglimite:
                color = 'red'
            else:
                color = 'orange'

            segments.append(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines+markers', line=dict(color=color), showlegend=False))

            with fig1.batch_update():
                fig1.data = []  # Efface les anciennes données
                fig1.add_traces(segments)
                

        t += time_step
        # Pause pour simuler le temps entre les mises à jour
        time.sleep(time_step)
        # await asyncio.sleep(time_step)
        # Timer(time_step, update_data).start()

def start_plot(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running
    running = True

    global etat_courant, Panne
    Panne = False
    etat_courant = "etat_Consom"

    # Démarrer la mise à jour des données
    # update_data()
    
    # asyncio.ensure_future(update_data())

    # # Démarrer la mise à jour des données dans un nouveau thread ou processus
    import threading
    thread = threading.Thread(target=update_data)
    thread.start()


def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_courant, Mg, PFC
    global segments
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_courant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    initialize_message()
    with fig1.batch_update():
        fig1.data = []  # Efface les anciennes données
        segments = []
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)

def serie_plot(button):
    """Fonction pour passer à l'état de série."""
    global etat_courant
    etat_courant = "etat_Serie"

def panne_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Panne
    Panne = True
    etat_courant = "etat_Panne"


def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False

def Demandepoche_plot(button):
    """Fonction pour passer à l'état de poche."""
    global DemandePoche
    DemandePoche = True
    # update_message("Quand lancer la prochain poche ?")


def Ajoutpoche_plot(button):
    """Fonction pour passer à l'état de poche."""
    global AjoutPoche
    AjoutPoche = True
    # update_message("AJOUT DE LA POCHE")

def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start")
    reset_button = widgets.Button(description="Reset")
    stop_button = widgets.Button(description="Stop")
    Demandepoche_button = widgets.Button(description="DemandePoche")
    Ajoutpoche_button = widgets.Button(description="AjoutPoche")
    serie_button = widgets.Button(description="Série")
    panne_button = widgets.Button(description="Panne")

    start_button.on_click(start_plot)
    reset_button.on_click(reset_plot)
    stop_button.on_click(stop_plot)
    Ajoutpoche_button.on_click(Ajoutpoche_plot)
    Demandepoche_button.on_click(Demandepoche_plot)
    serie_button.on_click(serie_plot)
    panne_button.on_click(panne_plot)

    display(widgets.HBox([start_button, reset_button, stop_button, Demandepoche_button, Ajoutpoche_button, serie_button, panne_button]),message_output, 
            fig1, fig2)


def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig1, scatter1, fig2, scatter2
    fig1, scatter1, fig2, scatter2 = create_figures()
    create_buttons()
    initialize_message()

# Appel de la fonction principale pour démarrer l'application
main()

Output()

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '56527388-bbdc-4e11-94a6-4d77ae09809a'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '666d30df-1093-413a-a15f-119f04c5d267'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})

#### Test lecture

##### Autres

In [8]:

# écris un code qui lis sans arrêt dans un fichier txt pour effectuer des instructions
# Ce fichier peux être changer par l'utilisateur régulièrement




# Ok, merci. Maintenant on veut affiche en fonction de ce que l'on lit dans le fichier. Si l'on lit Option1 alors on affiche sin(t) , si l'on lit Option2 alors 2+t. 
# Le temps t varie a chaque fonction dans la boucle principale. On veut une unique figure dans tout le code. A chaque changement d'option, on calcule sin(t) ou 2+t et l'on rajoute dans la liste des données à afficher. On veut avoir un affichage continue dans une unique figure et graphique qui est mise à jour constamment  avec plotly.


# un peu comme ceci 

# def create_figures():
#     """Crée et configure les figures Plotly pour Mg et PFC."""
#     fig1 = go.FigureWidget()

#     # Un seul segment pour Mg, avec la couleur dynamique
#     scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    
#     fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")
    
#     fig2 = go.FigureWidget()
#     scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
#     fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

#     return fig1, scatter1, fig2, scatter2


# update_scatter(scatter, x_data, y_data) ne marche pas non plus


# Toujours aucune est tracer, peux- tu m'expliquer les raisons éventuels ?


# En rajoutant fig.show() après mise à jour, on a plusieurs figure crée, or  je souhaite conserver une unique figure !!




##### Parametres Globales à mettre dans fichier

In [9]:
# Paramètres sur la poche de traitement 
PPT = 1250 # en kg
S = 0.011 # en %
TPT = 1480 # en degrés Celsius
R = 95 # en %
masse_fil = 418 # en g/m
masse_mg_fil = 43 # en g/m
eP = 0.001  # en %


# Paramètres sur le Four de Coulée
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg

PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %

temps_serie = 5  # en min
temps_traitement = 10 # en min
temps_gs = 4 # en min





temps_serie_courant = temps_serie
Mglimite = 0.03 # en %





In [ ]:
import numpy as np
import plotly.graph_objs as go
import time
import os
from IPython.display import display


def PendantReset () :
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_suivant, Mg, PFC
    global segments
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_suivant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    global message
    message = "Aucun Message"

    with fig1.batch_update():
        fig1.data = []  # Efface les anciennes données
        segments = []
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)
    update_message(message)
    return 




# Graphiques : Création et mise à jour des figures
def create_plot(title, xaxis_title, yaxis_title):
    """Crée un graphique Plotly avec des axes configurés."""
    fig = go.FigureWidget()
    scatter = fig.add_scatter(mode='lines+markers').data[0]
    fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title)
    display(fig)
    return fig, scatter


def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig_Mg, scatter_Mg = create_plot("Consommation de Mg", "Temps", "Mg")
    fig_PFC, scatter_PFC = create_plot("Consommation de Fonte", "Temps", "Fonte")
    return fig_Mg, scatter_Mg, fig_PFC, scatter_PFC


# Pour le PFC
def update_plot_data(fig, scatter, xdata, ydata):
    """Met à jour les données d'un graphique Plotly."""
    with fig.batch_update():
        scatter.update(x=xdata, y=ydata)


# Pour le Mg
def add_colored_segment(fig, x0, x1, y0, y1, color):
    """Ajoute un segment coloré à un graphique."""
    segment = go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines+markers', line=dict(color=color), showlegend=False)
    with fig.batch_update():
        # fig.data = []  # Efface les anciennes données
        fig.add_trace(segment)

def determine_segment_color(Mg, PFC, Mgmin, PFCmax, PPT):
    """Détermine la couleur du segment en fonction des valeurs de Mg."""
    # global Mgmin, PFCmax, PPT

    if (PFC + PPT) >= PFCmax and Mg <= Mgmin :
        message = "On ne peut plus sauver la fonte GS, faites autre chose !"
        update_message(message)
        return 'red'
    elif  (PFC + PPT) <= PFCmax and Mg <= Mgmin:
        message = "Le Mg n'est pas bon mais on peut encore sauver le Mg."
        update_message(message)
        return 'orange'
    else:
        message = "Le Mg est bon."
        update_message(message)
        return 'green'
    

    

# Pour Mise à jour régulière de la figure
def update_figure(Mg, PFC,fig1, fig2):
    """Met à jour les données dans les figures Plotly."""
    update_plot_data(fig2, scatter2, timedata, PFCdata)

    if len(timedata) >= 2 :
        x0, x1 = timedata[-2], timedata[-1]
        y0, y1 = Mgdata[-2], Mgdata[-1]

        color = determine_segment_color(Mg, PFC)
        add_colored_segment(fig1, x0, x1, y0, y1, color)



def simulation_step(Mg, PFC, etat_courant, t, timedata, Mgdata, PFCdata):
    """
    Exécute une étape de simulation : met à jour les valeurs de Mg et PFC, 
    gère les états, et stocke les résultats.
    """
    # Variables dans ce programme
    # global t, timedata, Mgdata, PFCdata
    if t != 0:
        Mg, PFC = gerer_poche(Mg, PFC)
        Mg, PFC, etat_courant = gerer_etat(Mg, PFC, etat_courant)

        # Stockage des résultats
        timedata.append(t)
        Mgdata.append(Mg)
        PFCdata.append(PFC)
    
    return Mg, PFC, etat_courant




segments = []

##### Test Classe

In [1]:
import os
import pandas as pd

class FileManager:
    def __init__(self, param_file, button_file, disa_file):
        self.param_file = param_file
        self.button_file = button_file
        self.disa_file = disa_file
        self.last_modified_button = None
        self.last_modified_disa = None

        self.running = False
        self.Panne = False
        self.reset = False
        self.DemandePoche = False
        self.AjoutPoche = False
        self.etat_courant = "etat_Consom"

        self.BoutonsEtParamètres = {}
        self.ProgrammeDISA = {}
        self.parametres_generaux = {}


    def read_parametres_generaux(self):
        """Lit le fichier ParamètresGénéraux et retourne un dictionnaire avec les noms de variables et leurs valeurs."""
        try:
            with open(self.param_file, 'r', encoding='utf-8') as file:
                lines = file.read().strip().split('\n')
                variable_names = lines[0].split(';')
                variable_values = lines[1].split(';')
                self.parametres_generaux = {var: float(val.replace(',', '.')) for var, val in zip(variable_names, variable_values)}
        except FileNotFoundError:
            print(f"Le fichier {self.param_file} n'existe pas.")
        except Exception as e:
            print(f"Une erreur est survenue : {e}")

    def read_file_boutons_et_parametres(self):
        """Lit le fichier BoutonsEtParamètres et retourne un dictionnaire avec les valeurs."""
        try:
            with open(self.button_file, 'r', encoding='utf-8') as file:
                content = file.read().strip()
                lines = content.split('\n')
                
                boutons_line = lines[0].split(';')
                self.BoutonsEtParamètres['Boutons'] = boutons_line[1] if boutons_line[0] == "Boutons" else 'STOP'

                second_line_headers = lines[1].split(';')
                third_line_values = lines[2].split(';')
                
                # Utilisation de valeurs par défaut en cas d'absence de données
                self.BoutonsEtParamètres['PPT'] = float(third_line_values[0].replace(',', '.')) if second_line_headers[0] == "PPT" else 0.0
                self.BoutonsEtParamètres['TPT'] = float(third_line_values[1].replace(',', '.')) if second_line_headers[1] == "TPT" else 0.0
                self.BoutonsEtParamètres['S'] = float(third_line_values[2].replace(',', '.')) if second_line_headers[2] == "S" else 0.0
            return self.BoutonsEtParamètres
        except FileNotFoundError:
            print(f"Le fichier {self.button_file} n'existe pas.")
            return None
        except Exception as e:
            print(f"Une erreur est survenue : {e}")
            return None

    def read_programme_disa(self):
        """Lit le fichier ProgrammeDISA et retourne son contenu sous forme de dictionnaire."""
        try:
            df = pd.read_csv(self.disa_file, sep=';', encoding='utf-8', usecols=["Cadences", "Nb Moules", "Poids"])
            self.ProgrammeDISA = df.to_dict(orient='list')
  
        except FileNotFoundError:
            print(f"Le fichier {self.disa_file} n'existe pas.")
        except pd.errors.EmptyDataError:
            print("Le fichier ProgrammeDISA est vide.")
        except Exception as e:
            print(f"Une erreur est survenue : {e}")

    def check_for_updates(self):
        """Vérifie les mises à jour des fichiers et traite les changements si nécessaire."""
        if os.path.exists(self.button_file):
            current_button_mod_time = os.path.getmtime(self.button_file)
            if self.last_modified_button is None or current_button_mod_time > self.last_modified_button:
                self.last_modified_button = current_button_mod_time
                self.handle_buttons(self.read_file_boutons_et_parametres())

        if os.path.exists(self.disa_file):
            current_disa_mod_time = os.path.getmtime(self.disa_file)
            if self.last_modified_disa is None or current_disa_mod_time > self.last_modified_disa:
                self.last_modified_disa = current_disa_mod_time
                self.read_programme_disa()

    def handle_buttons(self, current_BoutonsEtParamètres):
        """Gère les changements d'état en fonction des boutons."""    
        print(f"Fichier BoutonsEtParamètres mis à jour : {current_BoutonsEtParamètres}")
        
        bouton = current_BoutonsEtParamètres['Boutons']
        
        if bouton == "START":
            self.running = True
            self.Panne = False
            self.etat_courant = "etat_Consom"
        
        elif bouton == "STOP":
            self.running = False
        
        elif bouton == "RESET":
            self.reset = True
        
        elif bouton == "DEMANDEPOCHE":
            self.DemandePoche = True

        elif bouton == "AJOUTPOCHE":
            self.AjoutPoche = True

        elif bouton == "SERIE":
            self.etat_courant = "etat_Serie"          

        elif bouton == "PANNE":
            self.Panne = True
            self.etat_courant = "etat_Panne"

        elif bouton == "FIN":
            self.etat_courant = "etat_Fin"


In [2]:
import plotly.graph_objects as go
from IPython.display import display
import time


class PlotManager:
    def __init__(self):
        self.fig_Mg, self.scatter_Mg = self.create_plot("Consommation de Mg", "Temps", "Mg")
        self.fig_PFC, self.scatter_PFC = self.create_plot("Consommation de Fonte", "Temps", "Fonte")

    def create_plot(self, title, xaxis_title, yaxis_title):
        fig = go.FigureWidget()
        scatter = fig.add_scatter(mode='lines+markers').data[0]
        fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title)
        display(fig)
        return fig, scatter

    def update_plot_data(self, fig, scatter, xdata, ydata):
        with fig.batch_update():
            scatter.update(x=xdata, y=ydata)

    def add_colored_segment(self, fig, x0, x1, y0, y1, color):
        segment = go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines+markers', line=dict(color=color), showlegend=False)
        with fig.batch_update():
            fig.add_trace(segment)

    def determine_segment_color(self, Mg, PFC, Mgmin, PFCmax, PPT):
        if (PFC + PPT) >= PFCmax and Mg <= Mgmin:
            return 'red'
        elif (PFC + PPT) <= PFCmax and Mg <= Mgmin:
            return 'orange'
        else:
            return 'green'

    def update_figure(self, Mg, PFC, timedata, Mgdata, PFCdata, Mgmin, PFCmax, PPT):
        self.update_plot_data(self.fig_PFC, self.scatter_PFC, timedata, PFCdata)
        if len(timedata) >= 2:
            x0, x1 = timedata[-2], timedata[-1]
            y0, y1 = Mgdata[-2], Mgdata[-1]
            color = self.determine_segment_color(Mg, PFC, Mgmin, PFCmax, PPT) 
            self.add_colored_segment(self.fig_Mg, x0, x1, y0, y1, color)


In [3]:
# Pour calculer la longueur du fil fourré 
def calcul_quantite_mg(P,S,t,e,T,R,Mg,K) :
    """
    Calcule la Quantité d'alliage au magnésium (en Kg) à introduire dans la fonte pour obtenir du graphite spherodial.
    Args:
    P: Poids de fonte à traiter en Kg.
    S: Taux de souffre de la fonte de base en %.
    t: Temps de séjour en minutes prévu pour la fonte après traitement.
    T: Température (degrés Celsius) de la fonte au moment du traitement, mesurée au couple.
    R: Rendement en magnésium de l'opération en %.
    Mg: Taux en magnésium dans l'alliage en %.
    K: Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal en %.

    Returns:
    Q: Quantité d'alliage au magnésium à utiliser en Kg.
    """
    Q = P * (0.76 * (S - 0.01) + K + t * e) * (T / 1450) ** 2 / (R * Mg / 100)

    return Q

def calcul_longueur_fil( PPT, S, temps_gs, TPT,
    R, masse_fil, masse_mg_fil, Mgmax, eP, 
    PFClimite, Mglimite ):

    # % de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    K = ( Mgmax*(PFClimite + PPT) - Mglimite*PFClimite )/ PPT

    # Masse de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    Mgfil = masse_mg_fil/masse_fil *100
    Q = calcul_quantite_mg(PPT,S,temps_gs,eP,TPT,R,Mgfil,K) # en Kg


    # Longueur de fil pour avoir la masse de Mg manquante
    L = Q / (masse_fil* 1e-3)   # en m

    return K, L




# Calcule du temps limite avant lancement prochain traitement
def calculer_temps_epuis_PFC(PFC, PFCmin, temps_serie, liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire):
    # Calcul de la fonte four consommable
    PFCconsommable = PFC - PFCmin

    # Initialisation du temps total
    temps_total = 0

    # Boucle sur chaque modèle de moules
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min
        
        # Fonte nécessaire pour produire tous les moules de ce modèle
        fonte_necessaire_i = quantite_moules_a_produire_i * masse_grappe_i

        if PFCconsommable >= fonte_necessaire_i:
            # Si la fonte est suffisante pour produire tous les moules
            temps_pour_produire_i = fonte_necessaire_i / cadence_fonte_i
            temps_total += temps_pour_produire_i
            PFCconsommable -= fonte_necessaire_i

            # Ajouter le temps de la série s'il y a d'autres modèles à produire
            if i < len(liste_masses_grappes_moules) - 1:
                temps_total += temps_serie
        else:
            # Si la fonte n'est pas suffisante, calcul du temps possible avec la fonte restante
            temps_possible = PFCconsommable / cadence_fonte_i
            temps_total += temps_possible
            break  # On arrête la boucle, car il n'y a plus de fonte

    return temps_total

def calcul_temps_limite(PFC, Mg, PFCmin, Mgmin, eC, temps_traitement, temps_serie, liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire):
    
    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre la fonte minimal (en Kg) dans le four de coulée
    temps_epuis_PFC = calculer_temps_epuis_PFC(PFC, PFCmin, temps_serie, liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire)
    delai_avt_traitement_fonte_four = temps_epuis_PFC - temps_traitement


    
    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre le pourcentage minimal (%) dans le four de coulée
    Mgconsommable = Mg - Mgmin
    temps_epuis_Mg = Mgconsommable/eC
    delai_avt_traitement_mg_four = temps_epuis_Mg - temps_traitement


    # Temps en minute avant de lancer le traitement (Du four de fusion au four de coulée)
    delai_avt_traitement = min(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)
    tempslimite = temps_traitement + delai_avt_traitement
    # mise a jour de la masse mg avant ajout poche après consomation de mg
    Mgconsommer = tempslimite*eC
    Mgconsommer = (temps_traitement + delai_avt_traitement)*eC
    Mglimite = Mg - Mgconsommer



    # Mise à jour de la masse de fonte avant ajout dans le four après consommation pendant
    # temps_traitement + delai_avt_traitement
    PFCconsommer = 0
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min

        # Vérification du temps restant par rapport au temps nécessaire pour produire tous les moules de ce modèle
        temps_pour_produire_i = quantite_moules_a_produire_i / (cadence_fonte_i / masse_grappe_i)
        
        if tempslimite > temps_pour_produire_i:
            # Si le délai est supérieur au temps nécessaire, consommer toute la fonte pour ce modèle
            PFCconsommer += cadence_fonte_i * temps_pour_produire_i
            tempslimite -= temps_pour_produire_i

            if i < len(liste_masses_grappes_moules) - 1:
                tempslimite -= temps_serie
        else:
            # Sinon, consommer la fonte en fonction du temps restant
            PFCconsommer += cadence_fonte_i * tempslimite
            break

    PFClimite =  PFC - PFCconsommer


    return delai_avt_traitement, Mglimite, PFClimite


In [4]:


class Simulation:
    def __init__(self, param_file, button_file, disa_file):
        # FileManager s'occupe de la lecture et de la gestion des fichiers
        self.file_manager = FileManager(param_file, button_file, disa_file)

        # Initialisation du gestionnaire de plots
        self.plot_manager = PlotManager()

        # Variables de simulation
        self.Mg = 0.045  # Quantité initiale de magnésium
        self.PFC = 3500  # Poids de fonte coulée initial
        self.t = 0  # Temps de simulation
        self.timedata = []
        self.Mgdata = []
        self.PFCdata = []

    def initialize_simulation(self):
        """Initialise la simulation en lisant les fichiers nécessaires via FileManager."""
        self.file_manager.read_parametres_generaux()
        self.file_manager.read_programme_disa()
        self.file_manager.read_file_boutons_et_parametres()

        if self.file_manager.parametres_generaux  and self.file_manager.ProgrammeDISA and self.file_manager.BoutonsEtParamètres:
            print("Simulation initialisée avec succès.")
        else:
            print("Échec de l'initialisation de la simulation. Vérifiez les fichiers.")

    def step(self):
        """Effectue une étape de la simulation."""
        if self.t != 0:
            self.handle_poche()
            self.file_manager.etat_courant = self.handle_state(self.file_manager.etat_courant)
            self.record_data()
            Mgmin = self.file_manager.parametres_generaux['Mgmin']
            PFCmax = self.file_manager.parametres_generaux['PFCmax']
            PPT = self.file_manager.BoutonsEtParamètres['PPT']
            self.plot_manager.update_figure(self.Mg, self.PFC, self.timedata, self.Mgdata, self.PFCdata, Mgmin, PFCmax, PPT)
        self.t += 1

    def run_simulation(self, interval=1):
        """Démarre la simulation après initialisation."""
        self.file_manager.read_parametres_generaux()
        self.initialize_simulation()

        while self.file_manager.etat_courant != "etat_Fin":
            try:
                self.file_manager.check_for_updates()
                if self.file_manager.running:
                    self.step()
            except FileNotFoundError as e:
                print(f"Erreur : {e}")
            except Exception as e:
                print(f"Une erreur est survenue : {e}")
            time.sleep(interval)

    def handle_poche(self):
        """Gère les événements liés à la poche de magnésium."""
        if self.file_manager.DemandePoche:
            self.handle_message("DemandePoche")
            self.file_manager.DemandePoche = False
        if self.file_manager.AjoutPoche:
            self.Mg = self.file_manager.parametres_generaux["Mgmax"]
            self.PFC += self.file_manager.BoutonsEtParamètres["PPT"]
            self.handle_message("AjoutPoche")
            self.file_manager.AjoutPoche = False

    def handle_message(self, event):
        """Gère l'affichage des messages selon l'événement fourni."""
        if event == "DemandePoche":

            liste_quantites_moules_a_produire = self.file_manager.ProgrammeDISA["Nb Moules"]
            liste_masses_grappes_moules = self.file_manager.ProgrammeDISA["Poids"]
            liste_cadences_moule_par_heure = self.file_manager.ProgrammeDISA["Cadences"]
            

            R = self.file_manager.parametres_generaux['R']
            masse_fil = self.file_manager.parametres_generaux['masse_fil']
            masse_mg_fil = self.file_manager.parametres_generaux['masse_mg_fil']
            PFCmin = self.file_manager.parametres_generaux['PFCmin']
            Mgmin = self.file_manager.parametres_generaux['Mgmin']
            Mgmax = self.file_manager.parametres_generaux['Mgmax']
            eC = self.file_manager.parametres_generaux['eC']
            eP = self.file_manager.parametres_generaux['eP']
            temps_traitement = self.file_manager.parametres_generaux['temps_traitement']
            temps_gs = self.file_manager.parametres_generaux['temps_gs']
            temps_serie = self.file_manager.parametres_generaux['temps_serie']


            PPT = self.file_manager.BoutonsEtParamètres['PPT']
            S = self.file_manager.BoutonsEtParamètres['S']
            TPT = self.file_manager.BoutonsEtParamètres['TPT']



            delai_avt_traitement, Mglimite, PFClimite = calcul_temps_limite(self.PFC, self.Mg, PFCmin, Mgmin, eC, temps_traitement, temps_serie, liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire)
            K, L = calcul_longueur_fil(PPT, S, temps_gs, TPT, R, masse_fil, 
                                       masse_mg_fil, Mgmax, eP, PFClimite, Mglimite)
            

            message = (f"Lancer la poche dans {delai_avt_traitement} minutes avec cette longueur "
                       f"de fil fourée {L} m pour une visée de {K} en %")
        elif event == "AjoutPoche":
            message = "On a ajouté la poche maintenant"
        else:
            message = "Aucun message"
        
        self.update_message(message)


    def pendant_consommation(self, Mg, PFC):
        # Constantes dans ce programme
        eC = self.file_manager.parametres_generaux['eC']

        # Variables dans ce programme
        liste_quantites_moules_a_produire = self.file_manager.ProgrammeDISA["Nb Moules"]
        liste_masses_grappes_moules = self.file_manager.ProgrammeDISA["Poids"]
        liste_cadences_moule_par_heure = self.file_manager.ProgrammeDISA["Cadences"]

        global message

        # On produit les moules !! 
        masse_grappe_i = liste_masses_grappes_moules[0]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
        
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
        
        # Quantités de moules du i-ème modèle produits en une minute
        # cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min
        cadence_moule_i = cadence_fonte_i / masse_grappe_i # en unités/min
        
        # Mise à jour du pourcentage de Mg et du poids fonte coulée
        PFC -= cadence_fonte_i
        Mg -= eC

        # On met à jour le programme de productions des moules
        liste_quantites_moules_a_produire[0] -= cadence_moule_i

        if liste_quantites_moules_a_produire[0] <= 0 :
            message ="On a théoriquement finis de produire ce modèle, passons au modèle suivant !"

        # On reste dans l'état consom tant que l'utilisateur ne dit pas de changer de Série
        etat_suivant= "etat_Consom"

        # Si on a fini de tout produire alors on stoppe la procédure 
        # On passe à l'état Fin
        if not liste_masses_grappes_moules:
            etat_suivant = "etat_Fin"
        
        return Mg, PFC, etat_suivant

    def pendant_changement_serie(self, Mg, PFC):
        # Constantes dans ce programme
        eC = self.file_manager.parametres_generaux['eC']
        
        # Variables dans ce programme
        temps_serie = self.file_manager.parametres_generaux['temps_serie']
        liste_quantites_moules_a_produire = self.file_manager.ProgrammeDISA["Nb Moules"]
        liste_masses_grappes_moules = self.file_manager.ProgrammeDISA["Poids"]
        liste_cadences_moule_par_heure = self.file_manager.ProgrammeDISA["Cadences"]


        temps_serie_courant = self.file_manager.parametres_generaux['temps_serie_courant']

        global message


        if temps_serie_courant == temps_serie :

            # Suppression des éléments déjà traités dans les listes
            liste_masses_grappes_moules.pop(0)
            liste_quantites_moules_a_produire.pop(0)
            liste_cadences_moule_par_heure.pop(0)
            
        # pendant Temps_Serie :
        Mg -= eC # perte de Mg par minute ou par seconde
        temps_serie_courant -= 1 # pas de temps en minute
       
        etat_suivant = "etat_Serie"


        if temps_serie_courant == 0:
            temps_serie_courant = temps_serie  # Réinitialiser le temps de série
            

            message ="On a finis le changement de modèle, commençons le prochain modèle !"

            etat_suivant = "etat_Consom"


        self.file_manager.parametres_generaux['temps_serie_courant'] = temps_serie_courant
        
        return Mg, PFC, etat_suivant

    def pendant_panne(self, Mg, PFC):
        # Variables dans ce programme
        Panne =  self.file_manager.Panne

        # Constantes dans ce programme
        eC = self.file_manager.parametres_generaux['eC']


        # pendant Temps_Panne non définis :
        Mg -= eC # perte de Mg par minute ou par seconde


        etat_suivant = "etat_Panne"
        if not Panne :
            etat_suivant = "etat_Consom"
        
        return Mg, PFC, etat_suivant

    def handle_state(self, etat_courant):
        """Gère les transitions d'états selon l'état courant."""
        if etat_courant == "etat_Consom":
            self.Mg, self.PFC, etat_suivant = self.pendant_consommation(self.Mg, self.PFC)
        elif etat_courant == "etat_Serie":
            self.Mg, self.PFC, etat_suivant = self.pendant_changement_serie(self.Mg, self.PFC)
        elif etat_courant == "etat_Panne":
            self.Mg, self.PFC, etat_suivant = self.pendant_panne(self.Mg, self.PFC)
        elif etat_courant == "etat_Fin":
            self.file_manager.running = False
            etat_suivant = "etat_Fin"
        return etat_suivant

    def record_data(self):
        """Enregistre les données de simulation pour suivi et affichage."""
        self.timedata.append(self.t)
        self.Mgdata.append(self.Mg)
        self.PFCdata.append(self.PFC)



    def update_message(self, message):
        """Affiche ou enregistre le message pour le suivi utilisateur."""
        print(message)  # À personnaliser selon votre affichage (widgets, logs, etc.)


# Test de la simulation
if __name__ == "__main__":
    chemin_ParamètresGénéraux = 'ParamètresGénéraux.CSV'
    chemin_BoutonsEtParamètres = 'BoutonsEtParamètres.CSV'
    chemin_ProgrammeDISA = 'ProgrammeDISA.CSV'

    sim = Simulation(chemin_ParamètresGénéraux, chemin_BoutonsEtParamètres, chemin_ProgrammeDISA)
    sim.run_simulation(1)  # Exécute la simulation pour 100 étapes


FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': 'affba962-1abe-4673-8273-469c8ca4cf15'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '5f157deb-a893-49c9-a834-4c31d3761e3f'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})

Simulation initialisée avec succès.
Fichier BoutonsEtParamètres mis à jour : {'Boutons': 'FIN', 'PPT': 1250.0, 'TPT': 1430.0, 'S': 0.011}


##### Fonctions pour concernant les 4 etat du FOUR de COULEE

In [11]:
# Pour le passage d'un etat à l'autre (consom, Serie, Panne, Fin)
def PendantChangement_serie ( Mg, PFC) :
    # Variables dans ce programme
    global temps_serie, temps_serie_courant
    global message



    if temps_serie_courant == temps_serie :

        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)
        
    # pendant Temps_Serie :
    Mg -= eC # perte de Mg par minute ou par seconde
    temps_serie_courant -= 1 # pas de temps en minute
    
    etat_suivant = "etat_Serie"


    if temps_serie_courant == 0:
        temps_serie_courant = temps_serie  # Réinitialiser le temps de série
        
        # # Suppression des éléments déjà traités dans les listes
        # liste_masses_grappes_moules.pop(0)
        # liste_quantites_moules_a_produire.pop(0)
        # liste_cadences_moule_par_heure.pop(0)

        message ="On a finis le changement de modèle, commençons le prochain modèle !"

        etat_suivant = "etat_Consom"
    
    return Mg, PFC, etat_suivant

def PendantConsommation ( Mg, PFC):
    
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire
    global message

    # On produit les moules !! 
    masse_grappe_i = liste_masses_grappes_moules[0]
    cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
    quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
    
    # La cadence de consommation en kg/min du i-ème modèle
    cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
    
    # Quantités de moules du i-ème modèle produits en une minute
    # cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min
    cadence_moule_i = cadence_fonte_i / masse_grappe_i # en unités/min
    
    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC -= cadence_fonte_i
    Mg -= eC

    # On met à jour le programme de productions des moules
    liste_quantites_moules_a_produire[0] -= cadence_moule_i

    if liste_quantites_moules_a_produire[0] <= 0 :
        message ="On a théoriquement finis de produire ce modèle, passons au modèle suivant !"

    # On reste dans l'état consom tant que l'utilisateur ne dit pas de changer de Série
    etat_suivant= "etat_Consom"

    # Si on a fini de tout produire alors on stoppe la procédure 
    # On passe à l'état Fin
    if not liste_masses_grappes_moules:
        etat_suivant = "etat_Fin"
    
    return Mg, PFC, etat_suivant

def PendantPanne ( Mg, PFC) :
    # Variables dans ce programme
    global Panne

    # pendant Temps_Panne non définis :
    Mg -= eC # perte de Mg par minute ou par seconde


    etat_suivant = "etat_Panne"
    if not Panne :
        etat_suivant = "etat_Consom"
    
    return Mg, PFC, etat_suivant




def gerer_etat(Mg, PFC, etat_courant):
    """
    Gère les transitions entre différents états et effectue les opérations associées.

    Parameters:
        etat_courant (str): L'état actuel du système ("etat_Consom", "etat_Serie", "etat_Fin").
        Mg (float): La quantité de matériau Mg.
        PFC (float): La quantité de PFC disponible.

    Returns:
        Tuple: Retourne les variables mises à jour pour Mg, PFC, etat_suivant.
    """

    # Constantes dans ce programme
    global running


    # Gérer l'état de consommation
    if etat_courant == "etat_Consom":
        Mg, PFC, etat_suivant = PendantConsommation ( Mg, PFC)

    # Gérer l'état de série
    elif etat_courant == "etat_Serie":
        Mg, PFC, etat_suivant = PendantChangement_serie ( Mg, PFC)

    # Gérer l'état de panne
    elif etat_courant == "etat_Panne":
        Mg, PFC, etat_suivant = PendantPanne ( Mg, PFC)

    # Gérer l'état de fin
    elif etat_courant == "etat_Fin":
        running = False  # Fin du processus
        etat_suivant = "etat_Fin"
    return Mg, PFC, etat_suivant



##### Fonctions pour concernant la gestion des poches

In [12]:


# Pour calculer la longueur du fil fourré 
def calcul_quantite_mg(P,S,t,e,T,R,Mg,K) :
    """
    Calcule la Quantité d'alliage au magnésium (en Kg) à introduire dans la fonte pour obtenir du graphite spherodial.
    Args:
    P: Poids de fonte à traiter en Kg.
    S: Taux de souffre de la fonte de base en %.
    t: Temps de séjour en minutes prévu pour la fonte après traitement.
    T: Température (degrés Celsius) de la fonte au moment du traitement, mesurée au couple.
    R: Rendement en magnésium de l'opération en %.
    Mg: Taux en magnésium dans l'alliage en %.
    K: Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal en %.

    Returns:
    Q: Quantité d'alliage au magnésium à utiliser en Kg.
    """
    Q = P * (0.76 * (S - 0.01) + K + t * e) * (T / 1450) ** 2 / (R * Mg / 100)

    return Q

def calcul_longueur_fil( PPT, S, TPT,
    R, masse_fil, masse_mg_fil, Mgmax, eP, 
    PFClimite, Mglimite ):

    # % de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    K = ( Mgmax*(PFClimite + PPT) - Mglimite*PFClimite )/ PPT

    # Masse de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    Mgfil = masse_mg_fil/masse_fil *100
    Q = calcul_quantite_mg(PPT,S,temps_gs,eP,TPT,R,Mgfil,K) # en Kg


    # Longueur de fil pour avoir la masse de Mg manquante
    L = Q / (masse_fil* 1e-3)   # en m

    return K, L


# Calcule du temps limite avant lancement prochain traitement
def calculer_temps_epuis_PFC(PFC):
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, temps_serie
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire, temps_serie

    # Calcul de la fonte four consommable
    PFCconsommable = PFC - PFCmin

    # Initialisation du temps total
    temps_total = 0

    # Boucle sur chaque modèle de moules
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min
        
        # Fonte nécessaire pour produire tous les moules de ce modèle
        fonte_necessaire_i = quantite_moules_a_produire_i * masse_grappe_i

        if PFCconsommable >= fonte_necessaire_i:
            # Si la fonte est suffisante pour produire tous les moules
            temps_pour_produire_i = fonte_necessaire_i / cadence_fonte_i
            temps_total += temps_pour_produire_i
            PFCconsommable -= fonte_necessaire_i

            # Ajouter le temps de la série s'il y a d'autres modèles à produire
            if i < len(liste_masses_grappes_moules) - 1:
                temps_total += temps_serie
        else:
            # Si la fonte n'est pas suffisante, calcul du temps possible avec la fonte restante
            temps_possible = PFCconsommable / cadence_fonte_i
            temps_total += temps_possible
            break  # On arrête la boucle, car il n'y a plus de fonte

    return temps_total

def calcul_temps_limite(PFC, Mg):
    
    global Mgmin, eC, temps_traitement

    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre la fonte minimal (en Kg) dans le four de coulée
    temps_epuis_PFC = calculer_temps_epuis_PFC(PFC)
    delai_avt_traitement_fonte_four = temps_epuis_PFC - temps_traitement


    
    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre le pourcentage minimal (%) dans le four de coulée
    Mgconsommable = Mg - Mgmin
    temps_epuis_Mg = Mgconsommable/eC
    delai_avt_traitement_mg_four = temps_epuis_Mg - temps_traitement


    # Temps en minute avant de lancer le traitement (Du four de fusion au four de coulée)
    delai_avt_traitement = min(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)
    tempslimite = temps_traitement + delai_avt_traitement
    # mise a jour de la masse mg avant ajout poche après consomation de mg
    Mgconsommer = tempslimite*eC
    Mgconsommer = (temps_traitement + delai_avt_traitement)*eC
    Mglimite = Mg - Mgconsommer



    # Mise à jour de la masse de fonte avant ajout dans le four après consommation pendant
    # temps_traitement + delai_avt_traitement
    PFCconsommer = 0
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min

        # Vérification du temps restant par rapport au temps nécessaire pour produire tous les moules de ce modèle
        temps_pour_produire_i = quantite_moules_a_produire_i / (cadence_fonte_i / masse_grappe_i)
        
        if tempslimite > temps_pour_produire_i:
            # Si le délai est supérieur au temps nécessaire, consommer toute la fonte pour ce modèle
            PFCconsommer += cadence_fonte_i * temps_pour_produire_i
            tempslimite -= temps_pour_produire_i

            if i < len(liste_masses_grappes_moules) - 1:
                tempslimite -= temps_serie
        else:
            # Sinon, consommer la fonte en fonction du temps restant
            PFCconsommer += cadence_fonte_i * tempslimite
            break

    PFClimite =  PFC - PFCconsommer


    return delai_avt_traitement, Mglimite, PFClimite







import ipywidgets as widgets

message_output = widgets.Output()
message = "Aucun Message"

def initialize_message():
    """Initialise le widget Output avec un fond blanc et pas de message."""
    with message_output:
        message_output.clear_output()
        display(widgets.HTML(
            "<div style='background-color:white; color:black; font-size:20px; padding:10px; text-align:center;'>"
            "Aucun message</div>"
        ))

    display(message_output)

def update_message(message):
    """Met à jour le message affiché avec un style personnalisé."""
    with message_output:
        message_output.clear_output()  # Efface les anciens messages
        display(widgets.HTML(
            f"<div style='background-color:yellow; color:red; font-size:24px; font-weight:bold; "
            f"border:2px solid black; padding:10px; text-align:center;'>"
            f"{message}</div>"
        ))




AjoutPoche, DemandePoche = False, False

def gerer_poche(Mg, PFC):
    """
    Gère l'ajout de la poche. Si Poche est True, met à jour Mg et PFC.
    
    Args:
    Mg (float): La quantité actuelle de Mg.
    PFC (float): La quantité actuelle de Fonte.

    
    Returns:
    Mg (float): La nouvelle quantité de Mg.
    PFC (float): La nouvelle quantité de fonte.
    """

    # Constantes dans ce programme
    global Mgmax, PPT
    # Variables dans ce programme
    global DemandePoche, AjoutPoche, message


    if DemandePoche :
        delai_avt_traitement, Mglimite, PFClimite = calcul_temps_limite(PFC, Mg)
        K, L = calcul_longueur_fil( PPT, S, TPT,
                R, masse_fil, masse_mg_fil, Mgmax, eP, 
                PFClimite, Mglimite )
        message = f"Lancer la poche dans {delai_avt_traitement} minutes avec cette longueur de fil fourée {L} m  pour une visée de {K} en %"
        DemandePoche = False

    if AjoutPoche:
        Mg = Mgmax
        PFC += PPT
        AjoutPoche = False
        DemandePoche = False
        message = f"On a ajouté la poche maintenant"

    
    # if not DemandePoche and not AjoutPoche :
    #     message = "Aucun Message"
    
    update_message(message)

    return Mg, PFC







def PendantReset () :
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_suivant, Mg, PFC
    global segments
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_suivant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    global message
    message = "Aucun Message"

    with fig1.batch_update():
        fig1.data = []  # Efface les anciennes données
        segments = []
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)
    update_message(message)
    return 

def PendantInitialisation () :
    """Fonction pour initialiser les données et le temps t."""
    global running, time_step, timedata, PFCdata, Mgdata
    global etat_courant, Mg, PFC, t
    global segments
    global DemandePoche, AjoutPoche, Panne
    timedata = []
    PFCdata = []
    Mgdata = []
    # etat_courant, etat_suivant = "etat_Consom", "etat_Consom"
    etat_courant = "etat_Consom"
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC
    t = 0

    message = "Aucun Message"
    update_message(message)
    return 





In [13]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, current_function, time_step, timedata, PFCdata, t
    running = False
    current_function = 0
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    t = initial_t

def create_figure():
    """Crée et configure la figure Plotly."""
    fig = go.FigureWidget()
    scatter = fig.add_scatter(mode='lines+markers').data[0]
    fig.update_layout(title="Courbe Interactive", xaxis_title="Temps", yaxis_title="Valeur")
    return fig, scatter

def update_data():
    """Fonction de mise à jour des données."""
    global running, current_function, time_step, timedata, PFCdata, t
    if running:
        timedata.append(t)
        if current_function == 0:
            PFCdata.append(np.sin(t))
        elif current_function == 1:
            PFCdata.append(2)
        
        with fig.batch_update():
            scatter.update(x=timedata, y=PFCdata)
        
        t += time_step

def start_or_next_function(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running, current_function
    current_function = (current_function + 1) % 2
    running = True

def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False

def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, t
    running = False
    timedata = []
    PFCdata = []
    t = 0
    with fig.batch_update():
        scatter.update(x=timedata, y=PFCdata)

def panne_plot(button):
    """Fonction pour signaler une panne."""
    global Panne
    Panne = True

def Finpanne_plot(button):
    """Fonction pour signaler la fin de la panne."""
    global Panne
    Panne = False

def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start/Next Function")
    stop_button = widgets.Button(description="Stop")
    reset_button = widgets.Button(description="Reset")

    panne_button = widgets.Button(description="Panne")
    Finpanne_button = widgets.Button(description="Fin Panne")
    
    start_button.on_click(start_or_next_function)
    stop_button.on_click(stop_plot)
    reset_button.on_click(reset_plot)
    panne_button.on_click(panne_plot)
    Finpanne_button.on_click(Finpanne_plot)

    display(widgets.HBox([start_button, stop_button, reset_button, panne_button, Finpanne_button]), fig)

def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig, scatter
    fig, scatter = create_figure()
    create_buttons()
    global running
    running = True
    # Boucle de mise à jour des données synchrone
    while running:
        update_data()
        time.sleep(time_step)  # Pause pour simuler l'écoulement du temps

# Appel de la fonction principale pour démarrer l'application
main()


FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': 'f3cc1ae6-0044-4e4d-96bf-054251dc8964'}],
    'layout': {'template': '...',
               'title': {'text': 'Courbe Interactive'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Valeur'}}}
})

KeyboardInterrupt: 

In [15]:
import numpy as np
import plotly.graph_objs as go
import time
import os
from IPython.display import display


# Fichier : Gestion de fichier
def read_file(filepath):
    """Lit le fichier et retourne son contenu."""
    with open(filepath, 'r') as file:
        return file.read().strip()


# Graphiques : Création et mise à jour des figures
def create_plot(title, xaxis_title, yaxis_title):
    """Crée un graphique Plotly avec des axes configurés."""
    fig = go.FigureWidget()
    scatter = fig.add_scatter(mode='lines+markers').data[0]
    fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title)
    display(fig)
    return fig, scatter


def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig_Mg, scatter_Mg = create_plot("Consommation de Mg", "Temps", "Mg")
    fig_PFC, scatter_PFC = create_plot("Consommation de Fonte", "Temps", "Fonte")
    return fig_Mg, scatter_Mg, fig_PFC, scatter_PFC


# Pour le PFC
def update_plot_data(fig, scatter, xdata, ydata):
    """Met à jour les données d'un graphique Plotly."""
    with fig.batch_update():
        scatter.update(x=xdata, y=ydata)


# Pour le Mg
def add_colored_segment(fig, x0, x1, y0, y1, color):
    """Ajoute un segment coloré à un graphique."""
    segment = go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines+markers', line=dict(color=color), showlegend=False)
    with fig.batch_update():
        # fig.data = []  # Efface les anciennes données
        fig.add_trace(segment)

def determine_segment_color(Mg, PFC):
    """Détermine la couleur du segment en fonction des valeurs de Mg."""
    global Mgmin, PFCmax, PPT

    if (PFC + PPT) >= PFCmax and Mg <= Mgmin :
        message = "On ne peut plus sauver la fonte GS, faites autre chose !"
        update_message(message)
        return 'red'
    elif  (PFC + PPT) <= PFCmax and Mg <= Mgmin:
        message = "Le Mg n'est pas bon mais on peut encore sauver le Mg."
        update_message(message)
        return 'orange'
    else:
        message = "Le Mg est bon."
        update_message(message)
        return 'green'
    

    

# Pour Mise à jour régulière de la figure
def update_figure(Mg, PFC,fig1, fig2):
    """Met à jour les données dans les figures Plotly."""
    update_plot_data(fig2, scatter2, timedata, PFCdata)

    if len(timedata) >= 2 :
        x0, x1 = timedata[-2], timedata[-1]
        y0, y1 = Mgdata[-2], Mgdata[-1]

        color = determine_segment_color(Mg, PFC)
        add_colored_segment(fig1, x0, x1, y0, y1, color)



def simulation_step(Mg, PFC, etat_courant):
    """
    Exécute une étape de simulation : met à jour les valeurs de Mg et PFC, 
    gère les états, et stocke les résultats.
    """
    # Variables dans ce programme
    global t, timedata, Mgdata, PFCdata
    if t != 0:
        Mg, PFC = gerer_poche(Mg, PFC)
        Mg, PFC, etat_courant = gerer_etat(Mg, PFC, etat_courant)

        # Stockage des résultats
        timedata.append(t)
        Mgdata.append(Mg)
        PFCdata.append(PFC)
    
    return Mg, PFC, etat_courant




segments = []
def monitor_file(filepath, fig1, fig2, interval=2):
    """Surveille les changements dans un fichier et met à jour les graphiques en fonction du contenu."""

    global running, time_step, timedata, PFCdata, Mgdata
    global etat_courant, Mg, PFC, t
    global segments
    global DemandePoche, AjoutPoche, Panne
    timedata = []
    PFCdata = []
    Mgdata = []
    etat_courant = "etat_Consom"
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC
    t = 0

    timedata.append(t)
    Mgdata.append(Mg)
    PFCdata.append(PFC)


    last_modified = None
    while etat_courant != "etat_Fin":
        try:
            # Vérifie la date de dernière modification
            current_modified = os.path.getmtime(filepath)
            
            # Si le fichier a été modifié, relit le fichier
            if last_modified is None or current_modified > last_modified:
                last_modified = current_modified
                current_content = read_file(filepath)
                print(f"Fichier mis à jour : {current_content}")

                if current_content == "START":
                    running = True

                    Panne = False
                    etat_courant = "etat_Consom"
                
                elif current_content == "STOP":
                    """Fonction pour arrêter la mise à jour."""
                    running = False
                
                elif current_content == "RESET":
                    PendantReset ()
                
                elif current_content == "DEMANDEPOCHE":
                    """Fonction pour passer à l'état de poche."""
                    DemandePoche = True

                elif current_content == "AJOUTPOCHE":
                    """Fonction pour passer à l'état de poche."""
                    AjoutPoche = True

                elif current_content == "SERIE":
                    """Fonction pour passer à l'état de série."""
                    etat_courant = "etat_Serie"          

                elif current_content == "PANNE":
                    """Fonction pour passer à l'état de panne."""
                    Panne = True
                    etat_courant = "etat_Panne"

                elif current_content == "FIN":
                    """Fonction pour passer à l'état de panne."""
                    etat_courant = "etat_Fin"     
                
                else:
                    break


            if running : # Pour pourvoir stopper la simulation
                Mg, PFC, etat_courant = simulation_step(Mg, PFC, etat_courant)

                print(liste_quantites_moules_a_produire)
                print(etat_courant)
                update_figure(Mg, PFC,fig1, fig2)
                
                t += 1

        except FileNotFoundError:
            print(f"Le fichier {filepath} n'existe pas.")
        except Exception as e:
            print(f"Une erreur est survenue : {e}")
        
        time.sleep(interval)

if __name__ == "__main__":
    filepath = "Boutons.txt"  # Spécifie le chemin du fichier à surveiller
    initialize_message()
    fig1, scatter1, fig2, scatter2 = create_figures()  # Crée les figures
    monitor_file(filepath, fig1, fig2)  # Met à jour les figures continuellement







# Peux-tu réorganiser ce code pour le rendre plus claire, lisible et facilement modifiable en particulier la fonction monitor_file?


Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': 'HTML(value="<div style=\'background-co…

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': 'ba887067-541f-4f72-9115-e694ecedb082'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '3b8f54a7-af55-43ca-8fb1-cc12429bb4be'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})

Fichier mis à jour : START
[121, 50, 35, 10, 100, 50, 35, 50, 40]
etat_Consom


[117.83333333333333, 50, 35, 10, 100, 50, 35, 50, 40]
etat_Consom


...

Fichier mis à jour : DEMANDEPOCHE


[21.500000000000007, 35, 10, 100, 50, 35, 50, 40]
etat_Consom


[5.666666666666675, 35, 10, 100, 50, 35, 50, 40]
etat_Consom


Fichier mis à jour : FIN


[5.666666666666675, 35, 10, 100, 50, 35, 50, 40]
etat_Fin
